# Building a Local Agent with Strands Agents and Ollama Model

This notebook demonstrates how to create a personal agent using Strands Agent and Ollama. The agent will be capable of performing various local tasks such as file operations, web searches, and system commands.

## What is Ollama?

[Ollama](https://ollama.com/) is an open-source framework that allows you to run large language models (LLMs) locally on your machine. It provides a simple API for interacting with these models, making it ideal for privacy-focused applications where you don't want to send data to external services.

Key benefits of Ollama:
- **Privacy**: All processing happens locally on your machine
- **No API costs**: Free to use as much as you want
- **Offline capability**: Works without internet connection
- **Customization**: Can be fine-tuned for specific use 


## Agent Details

<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |Ollama Model - to create a file operations agent   |
|Agent Structure     |Single agent architecture                          |


</div>


### Agent Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

In [1]:
!pip install -r requirements.txt

## Setup and Installation

Before running this notebook, make sure you have:

1. Install Ollama: Follow instructions at [https://ollama.com/download](https://ollama.com/download)
2. Start the Ollama server: `ollama serve`
3. Downloaded a model with Ollama: `ollama pull llama3.2:1b`

Refer to the [documentation](https://cuddly-sniffle-lrmk2y7.pages.github.io/0.1.x/user-guide/concepts/model-providers/ollama/) for detailed instructions.

In this notebook, we will download Ollama for the linux distribution for compatibility with SageMaker Studio. This is done for code execution during AWS lead workshops on Workshop Studio. If you are running this code locally, you should adjust the steps to download Ollama to your current enviroment.

In [2]:
# this will work on linux computers
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
import subprocess
subprocess.Popen(['ollama', 'serve'])

<Popen: returncode: None args: ['ollama', 'serve']>

In [4]:
!ollama pull llama3.2:1b

[GIN] 2025/05/29 - 22:30:11 | 200 |    2.695633ms |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ 

time=2025-05-29T22:30:11.724Z level=INFO source=routes.go:1206 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/sagemaker-user/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR

pulling manifest ⠙ [GIN] 2025/05/29 - 22:30:12 | 200 |  236.480865ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest 
pulling 74701a8c35f6: 100% ▕██████████████████▏ 1.3 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 4f659a1e86d7: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
# Import necessary libraries
import os

import requests

# Import strands components
from strands import Agent, tool
from strands.models.ollama import OllamaModel

In [6]:
# Check if Ollama is running:
try:
    response = requests.get("http://localhost:11434/api/tags")
    print("✅ Ollama is running. Available models:")
    for model in response.json().get("models", []):
        print(f"- {model['name']}")
except requests.exceptions.ConnectionError:
    print("❌ Ollama is not running. Please start Ollama before proceeding.")

[GIN] 2025/05/29 - 22:30:13 | 200 |     1.07215ms |       127.0.0.1 | GET      "/api/tags"
✅ Ollama is running. Available models:
- llama3.2:1b
- llama3.2:latest


## Defining Custom Tools

Tools are functions that the agent can use to interact with the environment. Below, we define several useful tools for our personal agent.

In [7]:
# File Operation Tools


@tool
def file_read(file_path: str) -> str:
    """Read a file and return its content.

    Args:
        file_path (str): Path to the file to read

    Returns:
        str: Content of the file

    Raises:
        FileNotFoundError: If the file doesn't exist
    """
    try:
        with open(file_path, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found."
    except Exception as e:
        return f"Error reading file: {str(e)}"


@tool
def file_write(file_path: str, content: str) -> str:
    """Write content to a file.

    Args:
        file_path (str): The path to the file
        content (str): The content to write to the file

    Returns:
        str: A message indicating success or failure
    """
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(os.path.abspath(file_path)), exist_ok=True)

        with open(file_path, "w") as file:
            file.write(content)
        return f"File '{file_path}' written successfully."
    except Exception as e:
        return f"Error writing to file: {str(e)}"


@tool
def list_directory(directory_path: str = ".") -> str:
    """List files and directories in the specified path.

    Args:
        directory_path (str): Path to the directory to list

    Returns:
        str: A formatted string listing all files and directories
    """
    try:
        items = os.listdir(directory_path)
        files = []
        directories = []

        for item in items:
            full_path = os.path.join(directory_path, item)
            if os.path.isdir(full_path):
                directories.append(f"Folder: {item}/")
            else:
                files.append(f"File: {item}")

        result = f"Contents of {os.path.abspath(directory_path)}:\n"
        result += (
            "\nDirectories:\n" + "\n".join(sorted(directories))
            if directories
            else "\nNo directories found."
        )
        result += (
            "\n\nFiles:\n" + "\n".join(sorted(files)) if files else "\nNo files found."
        )

        return result
    except Exception as e:
        return f"Error listing directory: {str(e)}"

## Creating the Ollama-powered Agent

Now we'll create our agent using the Ollama model and the tools we defined above.

Note: You can add more tools like `execute_commands`, `search_files` etc.

In [8]:
# Define a comprehensive system prompt for our agent
system_prompt = """
You are a helpful personal assistant capable of performing local file actions and simple tasks for the user.

Your key capabilities:
1. Read, understand, and summarize files.
2. Create and write to files.
3. List directory contents and provide information on the files.
4. Summarize text content

When using tools:
- Always verify file paths before operations
- Be careful with system commands
- Provide clear explanations of what you're doing
- If a task cannot be completed, explain why and suggest alternatives

Always be helpful, concise, and focus on addressing the user's needs efficiently.
"""

model_id = (
    "llama3.2:1b"  # You can change this to any model you have pulled with Ollama.
)

#### Configure the Ollama model
Make sure your Ollama service is running at http://localhost:11434 and your `model_id` is in the list of Ollama models printed above.

In [9]:
ollama_model = OllamaModel(
    model_id=model_id,
    host="http://localhost:11434",
    params={
        "max_tokens": 4096,  # Adjust based on your model's capabilities
        "temperature": 0.7,  # Lower for more deterministic responses, higher for more creative
        "top_p": 0.9,  # Nucleus sampling parameter
        "stream": True,  # Enable streaming responses
    },
)

# Create the agent
local_agent = Agent(
    system_prompt=system_prompt,
    model=ollama_model,
    tools=[file_read, file_write, list_directory],
)

## Testing the Agent

Let's test our agent with some example tasks.

In [10]:
local_agent(
    "Read the file in the path `sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf` and summarize it in 5 bullet points."
)

[GIN] 2025/05/29 - 22:30:13 | 500 |   84.301172ms |       127.0.0.1 | POST     "/api/chat"


time=2025-05-29T22:30:13.920Z level=INFO source=server.go:135 msg="system memory" total="3.8 GiB" free="1.5 GiB" free_swap="0 B"
time=2025-05-29T22:30:13.921Z level=WARN source=server.go:163 msg="model request too large for system" requested="2.1 GiB" available=1655562240 total="3.8 GiB" free="1.5 GiB" swap="0 B"
time=2025-05-29T22:30:13.921Z level=INFO source=sched.go:455 msg="NewLlamaServer failed" model=/home/sagemaker-user/.ollama/models/blobs/sha256-74701a8c35f6c8d9a4b91f3f3497643001d63e0c7a84e085bed452548fa88d45 error="model requires more system memory (2.1 GiB) than is available (1.5 GiB)"


ResponseError: model requires more system memory (2.1 GiB) than is available (1.5 GiB) (status code: 500)

In [ ]:
# Example 2: List files in the current directory
response = local_agent("Show me the files in the current directory")
print(response)

In [ ]:
# Example 3: Create a sample file
response = local_agent(
    "Create a file called 'sample.txt' with the content 'This is a test file created by my Ollama agent.'"
)
print(response)

In [ ]:
# Example 4: create a readme file after reading and understanding multiple files
response = local_agent("Create a readme.md for the current directory")
print(response)

## Conclusion

In this notebook, we've created a local personal agent using Stands and Ollama. The agent can perform file operations (read, write, append) and Summarize/Analyze text

This demonstrates the power of running AI models locally with Ollama, combined with the flexibility of strands' tool system. You can extend this agent by adding more tools or using different Ollama models based on your needs.

### Next Steps (Ideas)

- Try different Ollama models to see how they affect the agent's capabilities
- Add more complex tools like web search, email sending, or calendar integration
- Implement memory for the agent to remember past interactions
- Create a simple UI for interacting with your agent